## KP Lung CD8 T-Cell 10X 5' Immune Profiling Metadata & Clonotype Processing

### For clonotype heatmap metadata

It is important for us to know which clonotypes arose from multiple different mice. Based off of the hashing and VDJ data collected, the vast majority of the clonotypes are derived from a single mouse. A small subset of clonotypes derive from one or multiple mice, so we want to distinguish between the two. 

In [ ]:
import pandas as pd

In [ ]:
# input/output directory
today = date.today()
od = "Final_Analysis/" + today.strftime("%Y-%m-%d") + "/Metadata/"
md_path = od + today.strftime("%Y-%m-%d") + "_" + "md_doublets_out.csv"
md_path_pre = od + today.strftime("%Y-%m-%d") + "_" + "md_posthash.csv"

In [ ]:
clonotypes = singlets_md['raw_clonotype_id'].unique() # generates list of clonotypes

# set up framework/dataframes
mult_clonotypes = pd.DataFrame()
bad_clonotypes, double_clonotypes, tripleplus_clonotypes, good_clonotypes = list(), list(), list(), list()

# generate subsets
singlets_demux = singlets_md.loc[singlets_md['raw_clonotype_id']!= '0'] # look only at cells with VDJ
singlets_demuxHash = singlets_md.loc[singlets_md['demux']!= '0'] # look only at cells with hashing

# score each clonotype for # demux identities
for i in clonotypes:
    singlets_clone = singlets_demuxHash.loc[singlets_demuxHash['raw_clonotype_id']== i] # partition just cells in a given clonotype
    mice = singlets_clone['demux'].unique() # get number of mice in clonotype
    mice_num = len(np.unique(mice))
    if (mice_num ==1):
        good_clonotypes.append(i)
    if (mice_num > 1):
        bad_clonotypes.append(i)
        mult_clonotypes
    if (mice_num == 2):
        double_clonotypes.append(i)
    if (mice_num >= 3):
        tripleplus_clonotypes.append(i)

# summary dataframe
singlets_demux = singlets_demux.replace({'demux': '0'}, {'demux': 'unmapped'}, regex = True)
singlets_mult = singlets_demux.loc[singlets_demux['raw_clonotype_id'].isin(bad_clonotypes)] # get multi-clonotype ids
mult_clonotypes = pd.DataFrame(data = singlets_mult[['raw_clonotype_id', 'demux']].value_counts()) # get paired value counts
mult_clonotypes = mult_clonotypes.rename(columns = {0: "cell_number_detected"})

# convert everything to dataframe
bad_clonotypes = pd.DataFrame(data=bad_clonotypes)
double_clonotypes = pd.DataFrame(data=double_clonotypes)
tripleplus_clonotypes = pd.DataFrame(data=tripleplus_clonotypes)

# write everything to file
bad_clonotypes.to_csv((od + today.strftime("%Y-%m-%d") + "_" + "multipleMouse_clonotypes.csv"))
double_clonotypes.to_csv((od + today.strftime("%Y-%m-%d") + "_" + "twoMouse_clonotypes.csv"))
tripleplus_clonotypes.to_csv((od + today.strftime("%Y-%m-%d") + "_" + "threeOrMoreMouse_clonotypes.csv"))
mult_clonotypes.to_csv((od + today.strftime("%Y-%m-%d") + "_" + "multi_clonotypes_aggregate.csv"))

In [ ]:
singlets_md = pd.read_csv("/Users/amandacruz/Dropbox (MIT)/SIIN vs SIY 10X/Final_Analysis/2020-11-02/2020-11-02_md_pre.csv")

In [ ]:
sel_clonotypes = pd.read_csv("/Users/amandacruz/Dropbox (MIT)/SIIN vs SIY 10X/Final_Analysis/2021-01-07/heatmap_clonotypes.csv")

In [ ]:
sel_clonotypes = sel_clonotypes.rename(columns={"Unnamed: 0": "clonotype"})
sel_clonotypes = sel_clonotypes.set_index(['clonotype'])

In [ ]:
for i in sel_clonotypes.index:
    singlets_clone = singlets_demuxHash.loc[singlets_demuxHash['raw_clonotype_id']== i] # partition just cells in a given clonotype
    mice = singlets_clone['demux'].unique() # get number of mice in clonotype
    sel_clonotypes.at[i,'Mouse'] = mice[0]

In [ ]:
sel_clonotypes.to_csv((od + today.strftime("%Y-%m-%d") + "_" + "all_clonotypes_metadata.csv"))


In [ ]:
print("Number of clonotypes coming from multiple mice.. " + str(len(bad_clonotypes)-1))
print("Number of clonotypes coming from one mouse.. " + str(len(good_clonotypes)))
print("Number of clonotypes detected.. " + str(len(clonotypes)-1))